In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
train=pd.read_csv('Train.csv')
test=pd.read_csv('Test.csv')
sample=pd.read_csv('Sample_submission.csv')

In [14]:
px.scatter(y=train['feature_1'],x=train['feature_2'],color=train ['Class'],facet_col=train['Class'])

In [13]:
px.scatter(y=train['feature_1'],x=train['feature_3'],color=train ['Class'],facet_col=train['Class'])

In [12]:
px.scatter(y=train['feature_2'],x=train['feature_3'],color=train ['Class'],facet_col=train['Class'])

In [20]:
px.bar(x=train['Class'].value_counts().index,y=train['Class'].value_counts(),color=train['Class'].value_counts().index)

In [45]:
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier,StackingClassifier

In [4]:
X=train.drop(['Class'],axis=1)
y=train['Class']


In [5]:
from imblearn.under_sampling import RandomUnderSampler
Rand=RandomUnderSampler(random_state=0)
X_res, y_res = Rand.fit_sample(X,y.ravel())

In [35]:
from sklearn.metrics import accuracy_score,SCORERS,auc

In [10]:
from sklearn.model_selection import StratifiedKFold,cross_val_score,GridSearchCV
from sklearn.svm import SVC,LinearSVC
fold=StratifiedKFold(n_splits=5)

In [70]:
accu=[]
models=[]
for train_idx,test_idx in fold.split(X_res,y_res):
    X_train,X_val=X_res.iloc[train_idx],X_res.iloc[test_idx]
    y_train,y_test=y_res[train_idx],y_res[test_idx]    
    mo=[('random',RandomForestClassifier(n_estimators=128,max_depth=8)),('gbt',LGBMClassifier(n_estimators=128,max_depth=8,learning_rate=0.1))]
    mod=StackingClassifier(estimators=mo,final_estimator=LinearSVC())
    mod.fit(X_train,y_train)
    pred=mod.predict(X_val)
    acc=accuracy_score(y_test,pred)
    accu.append(acc)
    models.append(mod)

In [71]:
accu

[0.9137931034482759,
 0.8070175438596491,
 0.7719298245614035,
 0.9122807017543859,
 0.8421052631578947]

In [75]:
X.shape[1]

1558

In [72]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [110]:
model=Sequential()
model.add(Dense(X.shape[1],input_shape=(X.shape[1],),activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')

In [111]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_56 (Dense)             (None, 1558)              2428922   
_________________________________________________________________
dense_57 (Dense)             (None, 256)               399104    
_________________________________________________________________
dense_58 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_59 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_60 (Dense)             (None, 8)                 520       
_________________________________________________________________
dense_61 (Dense)             (None, 1)                 9         
Total params: 2,910,795
Trainable params: 2,910,795
Non-trainable params: 0
___________________________________________

In [113]:
early=EarlyStopping(monitor='val_loss',mode='min',verbose=2,patience=10)
model.fit(X_res,y_res,validation_split=0.3,epochs=50)

Epoch 1/50
7/7 [==============================] - 0s 32ms/step - loss: 0.3327 - accuracy: 0.8700 - val_loss: 0.5701 - val_accuracy: 0.7907
Epoch 2/50
7/7 [==============================] - 0s 25ms/step - loss: 0.2934 - accuracy: 0.8800 - val_loss: 0.7675 - val_accuracy: 0.7326
Epoch 3/50
7/7 [==============================] - 0s 27ms/step - loss: 0.2468 - accuracy: 0.9100 - val_loss: 0.7993 - val_accuracy: 0.6744
Epoch 4/50
7/7 [==============================] - 0s 26ms/step - loss: 0.1835 - accuracy: 0.9450 - val_loss: 2.2351 - val_accuracy: 0.2326
Epoch 5/50
7/7 [==============================] - 0s 25ms/step - loss: 0.2479 - accuracy: 0.9200 - val_loss: 1.1799 - val_accuracy: 0.5233
Epoch 6/50
7/7 [==============================] - 0s 29ms/step - loss: 0.1901 - accuracy: 0.9250 - val_loss: 1.8856 - val_accuracy: 0.3023
Epoch 7/50
7/7 [==============================] - 0s 24ms/step - loss: 0.2731 - accuracy: 0.8950 - val_loss: 1.3439 - val_accuracy: 0.4767
Epoch 8/50
7/7 [===========

In [114]:
p=model.predict_classes(test)

In [62]:
preds=[]
for mod in models:
    p=model.predict_proba(test)
    preds.append(np.array(p))

In [63]:
preds=np.array(preds)

In [64]:
preds1=sum(preds)

In [65]:
preds1=preds1/5

In [66]:
predictions=np.argmax(preds1,axis=1)

In [67]:
predictions

array([1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [117]:
sample['Class']=p

In [118]:
sample.to_csv('pred17.csv',index=False)